In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import random
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
import tensorflow as tf
import pandas as pd
from src import utils
from src.features.load_dataset import import_stamp_yoochoose
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import os
import shutil
import datetime

from tensorflow import keras
from tqdm.notebook import tqdm
from collections import defaultdict
import functools
import copy
from itertools import chain
from src.models import TestModel
from src.models import stmp
from src.metrics import MrrMetric

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(f"tensorflow version: {tf.__version__}")

tensorflow version: 2.2.0


In [2]:
train_data, val_data, test_data, idx2item, embedding_sizes = import_stamp_yoochoose(64)

X_train, y_train = train_data
X_val, y_val = val_data
X_test, y_test = test_data

num_x_train = len(X_train)
n_items, max_session_len = embedding_sizes
max_embedding_key = n_items

print(f"n_items: {n_items}")
print(f"max_session_len: {max_session_len}")

using last item in session as class
using last item in session as class
train_val_test_split @ 0.8 train, 0.2 val, 0 test
n_items: 17695
max_session_len: 146


In [8]:
# config = TestModel.default_config(n_items, max_session_len)
# model = TestModel.model(config=config)

config = stmp.default_config(n_items)
model = stmp.model(config=config)

train_metrics = {
#     "acc": tf.keras.metrics.SparseCategoricalAccuracy(),
#     "top5 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5),
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
#     "top20 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20),
#     "mrr5": MrrMetric(k=5),
#     "mrr10": MrrMetric(k=10),
#     "mrr20": MrrMetric(k=20)
}
val_metrics = {
#     "acc": tf.keras.metrics.SparseCategoricalAccuracy(),
#     "top5 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5),
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
#     "top20 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20),
#     "mrr5": MrrMetric(k=5),
#     "mrr10": MrrMetric(k=10),
#     "mrr20": MrrMetric(k=20)
}
test_metrics = {
    "acc": tf.keras.metrics.SparseCategoricalAccuracy(),
    "top5 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5),
    "top10 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10),
    "top20 acc": tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20),
    "mrr5": MrrMetric(k=5),
    "mrr10": MrrMetric(k=10),
    "mrr20": MrrMetric(k=20)
}

train_metrics_rec = defaultdict(lambda: [])
val_metrics_rec = defaultdict(lambda: [])
test_metrics_rec = defaultdict(lambda: [])

current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
print(f"starting experiment '{current_time}'")
expr_dir = Path("logs")/current_time
train_log_dir = expr_dir/"train"
val_log_dir = expr_dir/"val"
test_log_dir = expr_dir/"test"

os.makedirs(expr_dir)
for tensorboard_dir in [train_log_dir, val_log_dir, test_log_dir]:
    os.makedirs(tensorboard_dir, exist_ok=True)

train_summary_writer = tf.summary.create_file_writer(str(train_log_dir))
val_summary_writer = tf.summary.create_file_writer(str(val_log_dir))
model_saver = utils.TopModelSaver("models/last_run_weights.h5py")

print(f"batch_size: {config['batch_size']}")
for epoch in range(config["epochs"]):
    print(f"epoch {epoch}")
    # Reset the metrics at the start of the next epoch
    for metric in utils.flatten([train_metrics.values(), val_metrics.values(), test_metrics.values()]):
        metric.reset_states()

    for i, (X, y) in tqdm(list(enumerate(utils.batchify(X_train, y_train, shuffle=True, batch_size=config["batch_size"])))):
        X = [sess[::-1] for sess in X] # reversing sessions so most recent item is last
        X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
        preds, loss = model.train_step(tf.constant(X), tf.constant(mask), tf.constant(y), train_metrics.values()) 
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', loss.numpy(), step=epoch*(int(num_x_train/config["batch_size"])+1)+i)

    for X, y in tqdm(list(utils.batchify(X_val, y_val, shuffle=True, batch_size=config["batch_size"]))):
        X = [sess[::-1] for sess in X]
        X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
        model.test_step(tf.constant(X), tf.constant(mask), tf.constant(y), val_metrics.values())

    with train_summary_writer.as_default():
        for label, metric in train_metrics.items():
            train_metrics_rec[label].append(metric.result())
            print(f"{label} train: {metric.result()}")
            tf.summary.scalar(label, metric.result(), step=epoch)
    with val_summary_writer.as_default():
        for label, metric in val_metrics.items():
            val_metrics_rec[label].append(metric.result())
            print(f"{label} val: {metric.result()}")
            tf.summary.scalar(label, metric.result(), step=epoch)

    model_saver.save_best(model, val_metrics["top10 acc"].result())

starting experiment '2020-07-07_11:23:18'
batch_size: 2048
epoch 0



top10 acc train: 0.05151473730802536
top10 acc val: 0.05038142204284668
new best score: 0.05038142204284668; saving weights @ models/last_run_weights.h5py
epoch 1



top10 acc train: 0.02234121784567833
top10 acc val: 0.01596948690712452
best score remains 0.05038142204284668; not saving weights
epoch 2



top10 acc train: 0.01626339927315712
top10 acc val: 0.016430547460913658
best score remains 0.05038142204284668; not saving weights
epoch 3



top10 acc train: 0.02301187254488468
top10 acc val: 0.026070918887853622
best score remains 0.05038142204284668; not saving weights
epoch 4


...


top10 acc train: 0.47645893692970276
top10 acc val: 0.23220722377300262
best score remains 0.24004526436328888; not saving weights
epoch 256



top10 acc train: 0.4772658348083496
top10 acc val: 0.23187190294265747
best score remains 0.24004526436328888; not saving weights
epoch 257



top10 acc train: 0.479801744222641
top10 acc val: 0.23111744225025177
best score remains 0.24004526436328888; not saving weights
epoch 258



top10 acc train: 0.47875383496284485
top10 acc val: 0.22772236168384552
best score remains 0.24004526436328888; not saving weights
epoch 259



top10 acc train: 0.4799484312534332
top10 acc val: 0.2331712692975998
best score remains 0.24004526436328888; not saving weights


In [14]:
model.load_weights(model_saver.location)

for metric in test_metrics.values():
    metric.reset_states()

preds, ys = [], []
for X, y in tqdm(list(utils.batchify(X_test, y_test, shuffle=True, batch_size=config["batch_size"]))):
    X = [sess[::-1] for sess in X]
    X, mask = utils.mask_length(X, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
    ys.extend(y)
    preds.extend(model.test_step(tf.constant(X), tf.constant(mask), tf.constant(y), test_metrics.values()).numpy())

# brute force verify acc
acc, acc5, acc10, acc20 = 0, 0, 0, 0
for y, pred in tqdm(list(zip(ys, preds))):
    argsorted_preds = (-pred).argsort()
    if y in argsorted_preds[:20]:
        acc20 += 1

        if y in argsorted_preds[:10]:
            acc10 += 1

            if y in argsorted_preds[:5]:
                acc5 += 1

                if y == argsorted_preds[0]:
                    acc += 1

print("brute force sanity check top20 acc", acc20/len(ys))
print("brute force sanity check top10 acc", acc10/len(ys))
print("brute force sanity check top5 acc", acc5/len(ys))
print("brute force sanity check acc", acc/len(ys))

for label, metric in test_metrics.items():
    print(label, metric.result())


brute force sanity check top20 acc 0.35010441138084053
brute force sanity check top10 acc 0.22500652571130253
brute force sanity check top5 acc 0.14056382145653876
brute force sanity check acc 0.0383059253458627
acc tf.Tensor(0.038240667, shape=(), dtype=float32)
top5 acc tf.Tensor(0.14402245, shape=(), dtype=float32)
top10 acc tf.Tensor(0.22820413, shape=(), dtype=float32)
top20 acc tf.Tensor(0.35297573, shape=(), dtype=float32)
mrr5 tf.Tensor(0.07290416, shape=(), dtype=float32)
mrr10 tf.Tensor(0.08403894, shape=(), dtype=float32)
mrr20 tf.Tensor(0.09257878, shape=(), dtype=float32)


In [201]:
a = np.arange(6).reshape(2, 3)
b = np.arange(6).reshape(2, 3)
c = np.arange(12).reshape(4, 3)

print(a)
print(b)
print(c)
print(utils.trilinear_product(a, b, c))

[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
tf.Tensor(
[[  9  24  39  54]
 [ 66 216 366 516]], shape=(2, 4), dtype=int64)


In [27]:
inputs = [[3, 1, 2, 3], [1, 2, 3]]
inputs = [np.array(sess) for sess in inputs]
padded_inputs, mask = utils.mask_length(inputs, justify="right")

X_embeddings = tf.nn.embedding_lookup(tf.constant([[0, 0], [1, 1], [2, 2], [3, 3]], dtype=tf.float32), padded_inputs)
X_s = X_embeddings[:, :-1]
ms = utils.masked_mean(X_s, mask[:, :-1, None])
mt = X_embeddings[:, -1]

print(ms)

tf.Tensor(
[[2 2]
 [1.5 1.5]], shape=(2, 2), dtype=float32)


In [26]:
sessions = [
    [[0], [1]],
    [[1], [2], [3]]
]
sessions = [np.array(sess) for sess in sessions]

utils.mask_length(sessions)

(array([[[0],
         [0],
         [0],
         [1]],
 
        [[0],
         [1],
         [2],
         [3]]]),
 array([[0, 0, 1, 1],
        [0, 1, 1, 1]], dtype=float32))

In [75]:
# config = TestModel.default_config(n_items=4, embedding_size=3, max_session_len=3)
# model = TestModel.model(config=config)

config = stmp.default_config(n_items=4, embedding_size=3)
model = stmp.model(config=config)

sessions = np.array([[1, 2, 3]], dtype=np.int32)
labels = np.array([1], dtype=np.int32)
sessions, mask = utils.mask_length(sessions, maskoff_vals=config["maskoff"], maskon_vals=config["maskon"], justify="right")
print("sessions", sessions)
print("mask", mask)

for i in range(1000):
    print("epoch", i)
    print("pred", model.train_step(tf.constant(sessions), tf.constant(mask), tf.constant(labels), [])[1])
    print("embs", model.item_embs)

sessions [[0 1 2 3]]
mask [[0 1 1 1]]
epoch 0
pred tf.Tensor(1.3862951, shape=(), dtype=float32)
embs <tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[1.67e-06, 2.54e-07, -7.72e-07],
       [0.000288, -0.00223, -0.00119],
       [0.000436, -0.00569, -0.00125],
       [-0.00156, -0.00288, 0.000633]], dtype=float32)>
epoch 1
pred tf.Tensor(1.3862956, shape=(), dtype=float32)
embs <tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[2.26e-06, 3.46e-07, -8.79e-07],
       [0.000281, -0.00223, -0.00119],
       [0.000439, -0.00569, -0.00125],
       [-0.00156, -0.00287, 0.00063]], dtype=float32)>
epoch 2
pred tf.Tensor(1.3862926, shape=(), dtype=float32)
embs <tf.Variable 'Variable:0' shape=(4, 3) dtype=float32, numpy=
array([[2.61e-06, 4.02e-07, -7.42e-07],
       [0.000274, -0.00223, -0.00119],
       [0.000442, -0.00568, -0.00125],
       [-0.00156, -0.00287, 0.000627]], dtype=float32)>


In [52]:
preds = [[1, 2, 3, 4], [1, 2, 3], []]
preds = [np.array(sess, dtype=np.float32) for sess in preds]

padded_preds, mask = utils.mask_length(preds)
print(utils.masked_mean(padded_preds, mask, axis=1))

ValueError: could not broadcast input array from shape (0) into shape (4)

In [40]:
@tf.function()
def graph(a): # in tensorflow graph mode
    print(f"in eager mode: {tf.executing_eagerly()}")
    tf.print(a * 0)

def eager(a): # in tensorflow eager mode
    print(f"in eager mode: {tf.executing_eagerly()}")
    tf.print(a * 0)

graph(tf.constant(np.nan))
eager(tf.constant(np.nan))

# wow optimizations

in eager mode: False
0
in eager mode: True
nan


In [112]:
import os
os.mkdir("58_acc_76_top5_85_top50.tf_save")
model.save_weights("58_acc_76_top5_85_top50.tf_save/model")

AttributeError: 'NoneType' object has no attribute 'replace'

In [6]:
# record experiment scores
import datetime

with open("last_expr_name.txt", "r+") as last_expr_name:
    try:
        last_expr_name = int(last_expr_name.read())+1
    except:
        last_expr_name = 0
with open("last_expr_name.txt", "w") as expr_names:
    expr_names.write(str(last_expr_name))

experiment_name = f"expr{last_expr_name}_{datetime.date.today()}"

import os
expr_dir = Path("records")/experiment_name
(expr_dir/"model").mkdir(parents=True, exist_ok=True)

print("saving at", str(expr_dir/"model/model"))
# model.save_weights(str(expr_dir/"model/model"))

topn_labels = ["acc", "top5 acc", "top10 acc", "top20 acc", "top50 acc"]
train_accs = [train_accs_rec[label] for label in topn_labels]
test_accs = [test_accs_rec[label] for label in topn_labels]
for label, train_acc, test_acc in zip(topn_labels, train_accs, test_accs):
    kwargs = {
        "title": f"test {label}",
        "xlabel": "epoch",
        "ylabel": label
    }
    save_accs(expr_dir/"scores", [test_acc], kwargs)
    
    kwargs = {
        "title": f"train {label}",
        "xlabel": "epoch",
        "ylabel": label
    }
    save_accs(expr_dir/"scores", [train_acc], kwargs)
    
kwargs = {
    "title": f"test accs",
    "xlabel": "epoch",
    "ylabel": "acc",
    "legend": topn_labels
}
save_accs(expr_dir/"scores", test_accs, kwargs)


with open(expr_dir/"scores"/"raw_scores.pickle", "wb+") as raw_scores_handle:
    scores = {}
    for label in topn_labels:
        scores[f"train {label}"] = train_accs_rec[label]
        scores[f"test {label}"] = test_accs_rec[label]
    pickle.dump(scores, raw_scores_handle)
    


SyntaxError: invalid syntax (<ipython-input-6-08255273a46b>, line 55)